In [ ]:
#################################
# Fast Instagram Scraper v1.0.0
# Release: 22.11.2020
# GitHub: do-me
#################################

# install torpy and pandas before

from torpy.http.requests import TorRequests
import json
from pandas.io.json import json_normalize
import requests
import time
import pandas as pd
import datetime
from tqdm.notebook import tqdm

# scraping parameters

# main
location_id = 123456 
max_posts = 100000000 # maximum number of posts to scrape
out_dir = "/some/path/" # directory to save csv file

# advanced
max_requests = 10000 # maximum number of requests from one tor end node
wait_between_requests = 0 # time in seconds to wait for next requests adding up to normal execution time ~8 seconds
max_tor_renew = 10000 # maximum number of new tor sessions
run_number = "" # will be added to filename; useful for pausing and resuming, see comment in next cells

# just in the beginning: define empty variables
# IMPORTANT: when pausing (=interrupting jupyter) and resuming do not execute this cell! 
# just execute the main loop below as last_cursor and post_list will be in memory
post_list = []

# a cursor is an arbitrary hash to paginate through Instagram's posts
# when opening the first page with 50 results it contains a hash to load the next page and so on
# it's pretty much like a page number just that the numbers are not in order 
# for the first page there is no cursor, so leave it empty
last_cursor = "" 
this_cursor = ""

# help function
# returns right Instagram link
def ilink(this_locid = location_id, cursor=""):
    instalink = 'https://instagram.com/graphql/query/?query_hash=ac38b90f0f3981c42092016a37c59bf7&variables={"id":"' + str(this_locid) + '","first":50,"after":"'+ cursor +'"}'
    return instalink

total_posts = 0
# main scraping function
def torsession():
    global last_cursor, this_cursor, location_id, post_list, out_dir, run_number, total_posts
    
    with TorRequests() as tor_requests:
        with tor_requests.get_session() as sess, tqdm(total=2000) as pbar:
            print("Circuit built.") # conncection works
            i = 0
            
            while i < max_requests: # enter main loop
                print("Start iteration {}: {}".format(i,datetime.datetime.now()))

                # saves all posts as csv for every iteration, 50 at once
                pf = json_normalize(post_list)
                
                file_name = "{}{}{}.csv".format(out_dir, location_id, run_number)
                pf.to_csv(file_name, index=False)
                print("File saved.")
            
                time.sleep(wait_between_requests) # take a nap
                              
                try: 
                    ireq = sess.get(ilink(cursor = last_cursor)) # fire request
                    idata = ireq.json() # get data from page as json
                    
                except:
                    try:
                        print("Tor end node blocked. Last response: {}".format(ireq))
                    except:
                        print("Tor end node blocked.")
                    return # go back to main loop and get next session
                
                # access response json
                edge_location_to_media = idata["data"]["location"]["edge_location_to_media"]
                
                # if while scraping new posts appear, they will be considered!
                total_posts = edge_location_to_media["count"]
                pbar.total = total_posts
                pbar.refresh()
                
                ipage = edge_location_to_media["edges"] # get posts
                post_list.extend(ipage) # extend list with all posts (50 every time)
                pbar.update(len(ipage))
                print("Succesfully appended iteration: {}".format(i))
                
                if len(post_list) > max_posts:
                    print("Maximum number of posts scraped:{}".format(len(post_list)))
                    return "no_more_page"
                
                # return completely if no more page available (has_next_page: False)
                if not edge_location_to_media["page_info"]["has_next_page"]:
                    return "no_more_page"
                
                # just in case above doesn't work: compare this and last cursor
                this_cursor = edge_location_to_media["page_info"]["end_cursor"]
                if this_cursor == last_cursor:
                    return "no_more_page"

                last_cursor = this_cursor  
                
                # for long pause and jupyter shutdown: saves only the last cursor
                open("{}{}_last_cursor.txt".format(out_dir,location_id), 'a').write(last_cursor+"\n")
                # alternatively just print last_cursor for every iteration
                # print(last_cursor)
                
                i+=1   

# main loop
def scrape():
    ii = 0 
    while ii < max_tor_renew:
        print("Initiating tor session {}".format(ii))
        if torsession() == "no_more_page": 
            print("Mined {} from {} total posts.".format(len(post_list),total_posts))
            break
        ii += 1

In [ ]:
# scrape! 
scrape()

# Short Breaks
# for pausing just interrupt the jupyter notebook and after to resume only re-execute this cell 
# all variables will still be in memory and the script takes off where it left
# works only if jupyter server is not closed meanwhile!

In [ ]:
# Long Breaks
# just interrupt and close everything when jupyter or computer is shut down
# afterwards execute first cell (to reload all functions) and then this cell
# it will get the last cursor and add run_number to filename

# uncomment the following lines
# from pathlib import Path
# last_cursor = Path("{}{}_last_cursor.txt".format(out_dir,location_id)).read_text().split("\n")[-2] # reads last cursor
# run_number = int(datetime.datetime.now().timestamp()) # change to some index number or just leave the timestamp but watch out for duplicates!
# scrape()